# train on gcp

## required libs

In [6]:
import os
service_account_keyfile = "/home/minesh_1291/Minesh/Projects/_keys/my-kaggle-competitions-45419c5943cc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_keyfile

In [9]:
# !gcloud auth activate-service-account --key-file $service_account_keyfile

In [10]:
!gcloud config set account "minesh.1291@gmail.com"

Updated property [core/account].


In [11]:
!gsutil ls

gs://mkc_transfer/
gs://my-kaggle-competitions.appspot.com/
gs://staging.my-kaggle-competitions.appspot.com/


## Model Name & Version

In [6]:
MODEL_NAME = "birdsong_recognition"
MODEL_VERSION = "v1"
PKG_VERSION = "1.0"

PROJECT_ID = "my-kaggle-competitions"
BUCKET_NAME = "ml-model-mlm-test"
REGION = "us-central1"

MODEL_VERSION_NAME = MODEL_NAME+"_"+MODEL_VERSION
MODEL_VERSION_NAME

'jura_we8_v23'

In [7]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


## make folder & files

- [model_name].my_pipeline.py  : Pipeline components - Feature calculation + Normalization
- [model_name].train.py        : Train using Train Data, Pipeline and Model
- [model_name].predictor.py    : Predict using New Data, Pipeline and Model

- labeled_dataset_Xy.csv

## Export `labeled_dataset_Xy.csv` 

In [12]:
# df_sr_train.dtypes
# df_sr_train.label = df_sr_train.label.astype("str")

In [88]:
df_sr_train.label.value_counts()

-1    15058
 0     4023
 1     1253
 4     1056
 3     1045
 2     1042
 5      937
 6      855
 7      777
Name: label, dtype: int64

In [8]:
# df_sr_train.fillna(0).to_csv("labeled_dataset_Xy.csv")

In [15]:
# copy the feature matrix to google cloud
!date -Idate
!gsutil cp combined_time_tsfresh_features.pkl gs://ml-train/custom_pipeline/$MODEL_NAME/train/$(date -Idate)_1p/labeled_dataset_Xy.joblib

2020-05-29
/snap/google-cloud-sdk/133/lib/third_party/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
Copying file://combined_time_tsfresh_features.pkl [Content-Type=application/octet-stream]...
\ [1 files][138.2 MiB/138.2 MiB]                                                
Operation completed over 1 objects/138.2 MiB.                                    


In [ ]:
# list the items in the folder
!gsutil ls -lha gs://ml-train/custom_pipeline/$MODEL_NAME/train/$(date -Idate)_1

## test local train - check for errors in script

In [80]:
# !/home/minesh_1291/anaconda3/bin/pip install google-cloud-storage

In [76]:
#!/home/minesh_1291/anaconda3/bin/python -m audio_model.train --gcs_data_path "./labeled_dataset_Xy.csv" --gcs_model_path .

features.shape (26046, 52) 
target.shape (26046,)
Training the model...
Model was successfully uploaded.


### check output

In [77]:
pipeline = f.joblib.load("pipeline.joblib")

In [78]:
print(pipeline.best_score_)
print(pipeline.best_params_)

0.9072507061331331
{'clf__estimator__max_depth': 7, 'clf__estimator__min_samples_leaf': 3, 'clf__estimator__min_samples_split': 6, 'clf__estimator__n_estimators': 5}


In [79]:
df_grid_results = pd.DataFrame(pipeline.cv_results_).round(3)
df_grid_results = df_grid_results.sort_values(by='mean_test_score', ascending=False)
df_grid_results[["params","mean_test_score","std_test_score"]][:3]

,params,mean_test_score,std_test_score
1,"{'clf__estimator__max_depth': 7, 'clf__estimat...",0.907,0.007
0,"{'clf__estimator__max_depth': 5, 'clf__estimat...",0.887,0.007


## packaging 

In [10]:
%%writefile ./setup.py
from setuptools import setup, find_packages

REQUIRED_PACKAGES = ['dataclasses_json', "xgboost"]

setup(name="audio_model",
      packages=["audio_model"],
      package_dir={'audio_model':'audio_model'},
      version="23.0",
      install_requires=REQUIRED_PACKAGES,
     )

Overwriting ./setup.py


In [ ]:
!python setup.py sdist --formats=gztar

### submit job

In [19]:
job_suffix = "_1p"

In [ ]:
! gcloud ai-platform jobs submit training {MODEL_VERSION_NAME}_training_$(date +"%Y%m%d_%H%M%S") \
  --job-dir gs://ml-model-mlm-test/custom_pipeline/$MODEL_NAME/train/$(date -Idate)$job_suffix \
  --package-path ./jura_we8 \
  --module-name jura_we8.train \
  --project $PROJECT_ID \
  --region $REGION \
  --runtime-version 1.15 \
  --python-version 3.7 \
  --scale-tier CUSTOM \
  --master-machine-type n1-standard-64\
  -- \
  --gcs_data_path gs://ml-train/custom_pipeline/$MODEL_NAME/train/$(date -Idate)$job_suffix/labeled_dataset_Xy.joblib \
  --gcs_model_path gs://ml-train/custom_pipeline/$MODEL_NAME/train/$(date -Idate)$job_suffix/model_fast/grid.joblib

In [ ]:
!gsutil ls gs://ml-train/custom_pipeline/$MODEL_NAME/train/$(date -Idate)$job_suffix/model_fast/

In [ ]:
!date -Idate
!gsutil cp gs://ml-train/custom_pipeline/$MODEL_NAME/train/$(date -Idate)$job_suffix/model_fast/grid.joblib .

In [91]:
# https://cloud.google.com/ai-platform/training/docs/runtime-version-list

In [107]:
sys.executable

'/usr/bin/python3.6'

In [21]:
import xgboost
import joblib 
xgboost.__version__, joblib.__version__

('0.82', '0.14.1')

In [22]:
grid_res = joblib.load("grid.joblib")

In [23]:
print(grid_res.best_score_)
print(grid_res.best_params_)

0.6900367315758899
{'clf__max_depth': 7, 'clf__min_samples_leaf': 3, 'clf__min_samples_split': 6, 'clf__n_estimators': 100}
